In [ ]:
# https://github.com/David-Woroniuk/Historic_Crypto
#%pip install Historic-Crypto
# from Historic_Crypto import HistoricalData 
# from Historic_Crypto import Cryptocurrencies
# from Historic_Crypto import LiveCryptoData

import plotly.graph_objects as go
import pandas as pd
import numpy as np

# from  datetime import datetime
# from dateutil.relativedelta import relativedelta

# def last_month_date_str():
#     dt = datetime.now() - relativedelta(months=1)
#     return dt.strftime("%Y-%m-%d-%H-%M")

# last_month = last_month_date_str()
# # 60, 300, 900, 3600, 21600, 86400
#
# #create a csv file for testing 
# df = HistoricalData('BTC-USD',900 ,last_month).retrieve_data()
# df.to_csv("BTCUSD.csv")

"""
 The problem of maxima, it all depends on the considered order.  
 
 orderint, optional
 How many points on each side to use for the comparison to consider comparator(n, n+x) to be True.
"""
order=2

key_open = "open"
key_high = "high"
key_low  = "low"
key_close = "close"

key_williams_fractal_up = "williams_fractal_up"
key_williams_fractal_down = "williams fractal down"

df = pd.read_csv("BTCUSD.csv")
df = df[0:100]

def williams_fractal_up(df: pd.DataFrame, key, period: int = order) -> pd.Series:
    periods = [p for p in range(-period, period + 1) if p != 0] # default [-2, -1, 1, 2]
    highs = [df[key] > df[key].shift(p) for p in periods]
    return pd.Series(np.logical_and.reduce(highs), index=df.index)

def williams_fractal_down(df: pd.DataFrame, key, period: int = order) -> pd.Series:
    periods = [p for p in range(-period, period + 1) if p != 0] # default [-2, -1, 1, 2]
    highs = [df[key] < df[key].shift(p) for p in periods]
    return pd.Series(np.logical_and.reduce(highs), index=df.index)

df[key_williams_fractal_up] = williams_fractal_up(df, key_high)
df[key_williams_fractal_down] = williams_fractal_down(df, key_low)

fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df[key_open], 
    high=df[key_high],
    low=df[key_low],  
    close=df[key_close]
)])

def draw_williams_fractals(fig, df):
    for index, row in df.iterrows():
        if row[key_williams_fractal_up]:
            fig.add_annotation(
                x=index,
                y=(df[key_high][index] + 80) ,
                xref="x",
                yref="y",
                ax=index,
                ay= (df[key_high][index] + 30),
                axref='x',
                ayref='y',
                text='',
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor='green'
            )

        if row[key_williams_fractal_down] :
            fig.add_annotation(
                x=index,
                y=(df[key_low][index] - 30) ,
                xref="x",
                yref="y",
                ax=index,
                ay= (df[key_low][index] - 80),
                axref='x',
                ayref='y',
                text='',
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor='red'
            )

from scipy import signal
indexes = signal.argrelextrema(df[key_low].values, np.less, order=order)
df["zigzag_low"] = False
df["zigzag_low"].iloc[indexes] = True

indexes = signal.argrelextrema(df[key_high].values, np.greater, order=order)
df["zigzag_high"] = False
df["zigzag_high"].iloc[indexes] = True

coords = [(0, df.loc[0, key_open])]
for index, row in df.iterrows():
    if row["zigzag_low"]:
        x = index
        y = row[key_low]
        coords.append((x,y,))

    if row["zigzag_high"]:
        x = index
        y = row[key_high]
        coords.append((x,y,))


for i in range(len(coords)):
    try:
        a = coords[i]
        b = coords[i+1]
    except IndexError:
        pass
    fig.add_shape(
        type='line',
        x0=a[0],
        y0=a[1],
        x1=b[0],
        y1=b[1],
        line=dict(color='Yellow',),
        xref='x',
        yref='y'
    )            
            
             
draw_williams_fractals(fig, df)
fig.update_layout(
    xaxis_rangeslider_visible=False, 
    plot_bgcolor='#000000'
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


C:\Users\thrynillan\AppData\Local\Temp\ipykernel_15196\3511897474.py:106: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thrynillan\AppData\Local\Temp\ipykernel_15196\3511897474.py:110: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

